In [38]:
from os import listdir
from os.path import isfile, join
import typing
from typing import List
import pandas as pd

In [39]:
data_path = "data/data-simulation/e-coli-100-node/"
tsv_files = [f for f in listdir(data_path) if f.endswith(".tsv")]

In [40]:
all_dfs = [pd.read_csv(join(data_path, f), sep="\t") for f in tsv_files]

In [41]:
merged_df = all_dfs[0]

Weird things happening with dataframe shape.

In [62]:
[print(idx, n, df.shape) for (n, (idx, df)) in zip(tsv_files, enumerate(all_dfs)) if df.shape[1] != 100]

7 Ecoli-1_proteins_dream4_timeseries.tsv (210, 101)
10 Ecoli-1_goldstandard_signed.tsv (119, 3)
14 Ecoli_goldstandard_signed.tsv (3647, 3)
16 Ecoli-1_nonoise_proteins_dream4_timeseries.tsv (210, 101)
18 Ecoli-1_dualknockouts_indexes.tsv (7, 2)
21 Ecoli-1_noexpnoise_proteins_dream4_timeseries.tsv (210, 101)
24 Ecoli-1_noexpnoise_dream4_timeseries.tsv (210, 101)
35 Ecoli_goldstandard.tsv (2447659, 3)
40 Ecoli-1_dream4_timeseries.tsv (210, 101)
42 Ecoli-1_goldstandard.tsv (9899, 3)
43 Ecoli-1_normalization_constant.tsv (0, 1)
44 Ecoli-1_nonoise_dream4_timeseries.tsv (210, 101)


[None, None, None, None, None, None, None, None, None, None, None, None]

In [78]:
all_dfs [10].head()

,asnC,asnA,+
0,asnC,mnmG,-
1,asnC,mioC,-
2,cbl,ssuB,+
3,cbl,tauD,+
4,cbl,tauA,+


So there is a time component. Might be fun for RNN models!?? But we should remove that.

In [ ]:
from pandas import DataFrame

In [50]:
merged_df.shape

(10, 100)

In [33]:
def check_columns_align(df_to_match: pd.DataFrame, list_of_dfs: List[pd.DataFrame]) -> List[bool]:
    match_bool = []
    

In [79]:
for df in all_dfs[1:]:
    if df.shape[1] != 100:
        if df.shape[1] == 101:
            merged_df = merged_df.append(df.iloc[:,1:], sort=True)
        else:
            pass
    else:
        merged_df = merged_df.append(df, sort=True)

In [82]:
save_path = join(data_path, "merged.csv")

In [83]:
merged_df.to_csv(save_path)